In [701]:
%pip install pandas
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [702]:
def getCsvs():
    stats = pd.read_csv("teamCsvFiles/OaklandAthletics.csv", encoding='latin1')
    stats.fillna(0, inplace=True)

In [703]:
folder_path = "teamCsvFiles/"
csvArray = []
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        csvArray.append({"teamName":file_name,"data":pd.read_csv(file_path, encoding='latin1')})

In [704]:
def formatPlayerStats(playerdf):
    game_scores = playerdf.groupby('gameId').agg(
    teamScore=('battingRuns', 'sum'),
    opponentScore=('pitchingRuns', 'sum')
)
    # Assign 'Win' or 'Loss' based on the comparison of scores
    game_scores['result'] = game_scores.apply(lambda row: 'Win' if row['teamScore'] > row['opponentScore'] else 'Loss', axis=1)
    # Merge the result back to the original dataframe based on gameId
    playerdf = playerdf.merge(game_scores['result'], left_on='gameId', right_index=True)
    playerdf["battingAvg"] = (playerdf["battingHits"]) / (stats["battingAtBats"])
    playerdf["onBasePercentage"] = (playerdf["battingHits"] + playerdf["battingBaseOnBalls"]) / (playerdf["battingAtBats"] + playerdf["battingBaseOnBalls"])
    playerdf.fillna(0, inplace=True) 
    return playerdf

In [705]:
def formatTeamStats(playerdf):
    aggdf = playerdf.groupby(['playerFullName']).sum().reset_index()
    # Drop columns
    aggdf = aggdf.drop(['gameId','opponentId','homeTeam?','playerInfoId'], axis=1)
    teamId = playerdf.iloc[0]["teamId"]  
    aggdf["teamId"] = teamId
    aggdf["battingAvg"] = (aggdf["battingHits"]) / (aggdf["battingAtBats"])
    aggdf["onBasePercentage"] = (aggdf["battingHits"] + aggdf["battingBaseOnBalls"]) / (aggdf["battingAtBats"] + aggdf["battingBaseOnBalls"])
    aggdf.fillna(0, inplace=True)
    return aggdf

In [706]:
teamStatsArray = []
for i in range(len(csvArray)):
    teamStatsArray.append({"teamName":csvArray[i]["teamName"],"data":formatTeamStats(csvArray[i]["data"])})

In [707]:
for i in range(len(csvArray)):
    csvArray[i]["data"] = formatPlayerStats(csvArray[i]["data"])

In [708]:
def addSabermetrics(df):
    df["battingSingles"] = (df["battingHits"] - df["battingDoubles"] - df["battingTriples"] - df["battingHomeRuns"])
    df["totalBases"] = (df["battingSingles"] + 2*df["battingDoubles"] + 3*df["battingTriples"] + 4*df["battingHomeRuns"])
    df["battingSlg"] = (df["totalBases"]/df["battingAtBats"])
    df["OBP+Slg"] = (df["onBasePercentage"] + df["battingSlg"])
    df["pitchingSingles"] = (df["pitchingHits"] - df["pitchingDoubles"] - df["pitchingTriples"] - df["pitchingHomeRuns"])
    df["pitchingRunsAllowedPerInning"] = (df["pitchingEarnedRuns"]/df["pitchingInningsPitched"])
    df["pitchingWinPercentage"] = (df["pitchingWins"] / (df["pitchingWins"] + df["pitchingLosses"]))
    df["pitchingWalks+Hits"] = (df["pitchingHits"] + df["pitchingBaseOnBalls"])
    df["pitchingWalks+HitsPercentage"] = ((df["pitchingHits"] + df["pitchingBaseOnBalls"]) / df["pitchingAtBats"])
    df["pitchingWalks+HitsPerInning"] = ((df["pitchingHits"] + df["pitchingBaseOnBalls"]) / df["pitchingInningsPitched"])
    df.fillna(0, inplace=True)
    

In [709]:
for i in range(len(csvArray)):
    addSabermetrics(csvArray[i]["data"])
    addSabermetrics(teamStatsArray[i]["data"])

In [726]:
for i in range(len(csvArray)):
    csvArray[i]["data"]["homeTeam?"] = csvArray[i]["data"]["homeTeam?"].map({
    True: 1,
    False: 0
})
    
    csvArray[i]["data"]["result"] = csvArray[i]["data"]["result"].map({
    "Win": 1,
    "Loss": 0
})

In [729]:
csvArray[0]["data"].columns

Index(['gameId', 'teamId', 'opponentId', 'homeTeam?', 'playerInfoId',
       'playerFullName', 'battingRuns', 'battingDoubles', 'battingTriples',
       'battingHomeRuns', 'battingStrikeOuts', 'battingBaseOnBalls',
       'battingHits', 'battingAtBats', 'battingStolenBases', 'battingRbi',
       'battingLeftOnBase', 'pitchingRuns', 'pitchingDoubles',
       'pitchingTriples', 'pitchingHomeRuns', 'pitchingStrikeOuts',
       'pitchingBaseOnBalls', 'pitchingHits', 'pitchingAtBats',
       'pitchingStolenBases', 'pitchingNumberOfPitches',
       'pitchingInningsPitched', 'pitchingWins', 'pitchingLosses',
       'pitchingHolds', 'pitchingBlownSaves', 'pitchingEarnedRuns',
       'pitchingPitchesThrown', 'result', 'battingAvg', 'onBasePercentage',
       'battingSingles', 'totalBases', 'battingSlg', 'OBP+Slg',
       'pitchingSingles', 'pitchingRunsAllowedPerInning',
       'pitchingWinPercentage', 'pitchingWalks+Hits',
       'pitchingWalks+HitsPercentage', 'pitchingWalks+HitsPerInning'],


In [747]:
features = csvArray[0]["data"].columns[1:5] 
features = features.append(csvArray[0]["data"].columns[6:])

In [748]:
features

Index(['teamId', 'opponentId', 'homeTeam?', 'playerInfoId', 'battingRuns',
       'battingDoubles', 'battingTriples', 'battingHomeRuns',
       'battingStrikeOuts', 'battingBaseOnBalls', 'battingHits',
       'battingAtBats', 'battingStolenBases', 'battingRbi',
       'battingLeftOnBase', 'pitchingRuns', 'pitchingDoubles',
       'pitchingTriples', 'pitchingHomeRuns', 'pitchingStrikeOuts',
       'pitchingBaseOnBalls', 'pitchingHits', 'pitchingAtBats',
       'pitchingStolenBases', 'pitchingNumberOfPitches',
       'pitchingInningsPitched', 'pitchingWins', 'pitchingLosses',
       'pitchingHolds', 'pitchingBlownSaves', 'pitchingEarnedRuns',
       'pitchingPitchesThrown', 'result', 'battingAvg', 'onBasePercentage',
       'battingSingles', 'totalBases', 'battingSlg', 'OBP+Slg',
       'pitchingSingles', 'pitchingRunsAllowedPerInning',
       'pitchingWinPercentage', 'pitchingWalks+Hits',
       'pitchingWalks+HitsPercentage', 'pitchingWalks+HitsPerInning'],
      dtype='object')